In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from keras.utils import np_utils

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [5]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.4 MB/s eta 0:00:00


In [7]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
#from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [8]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [9]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [10]:
Fs = 100

In [ ]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_50_subs.csv',header=None)

In [12]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/transfer_set_50_subs.csv',header=None)

In [ ]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [ ]:
x_transfer_set=transfer_df.drop([3000],axis=1)

In [ ]:
x_transfer_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.026093,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.032245,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.004273,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.079087,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143


In [ ]:
y_transfer=transfer_df[3000]

In [ ]:
y_transfer

0         4
1         3
2         2
3         2
4         0
         ..
116713    2
116714    4
116715    4
116716    2
116717    2
Name: 3000, Length: 116718, dtype: int64

In [ ]:
x_transfer_set_rshp=np.array(x_transfer_set).reshape(116718,3000,1)

In [ ]:
x_transfer_set_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.01339153],
        [ 0.03071675],
        [ 0.0298049 ],
        ...,
        [ 0.041659  ],
        [ 0.03299639],
        [ 0.02433378]],

       [[-0.0574574 ],
        [-0.06562874],
        [-0.10119811],
        ...,
        [-0.07091608],
        [-0.06851275],
        [-0.09014277]],

       [[-0.06574184],
        [-0.08269832],
        [-0.08636458],
        ...,
        [-0.15052423],
        [-0.08728115],
        [-0.04282768]]])

In [ ]:
y_transfer_=np_utils.to_categorical(y_transfer)

In [ ]:
y_transfer_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
subject_466_tst_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [13]:
subject_466_tst_df=pd.read_csv('/content/gdrive/My Drive/sub_466/x_test_sub.csv',header=None)

In [ ]:
subject_466_tst_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,-0.031299,-0.065233,0.074522,-0.027280,0.090150,-0.206774,0.006654,-0.084432,0.140158,-0.162570,...,-0.004062,0.038802,-0.030852,-0.022369,-0.033085,-0.023262,-0.080860,0.099526,-0.012992,0
1,-0.275376,-0.251674,-0.206680,-0.121914,-0.158472,-0.066073,-0.146821,-0.198645,-0.142804,-0.215920,...,0.036771,0.054849,0.039181,0.034361,0.014274,0.049627,0.029942,0.003829,-0.006616,0
2,0.115511,0.127965,0.159300,0.145641,0.166933,0.172959,0.168540,0.164522,0.164924,0.166531,...,0.022710,0.048421,0.068508,0.059268,0.057259,0.068106,0.115913,0.111895,0.114306,2
3,0.005034,0.014676,0.048823,0.050430,0.053242,0.070517,0.107878,0.115913,0.103057,0.079355,...,0.014274,0.003025,-0.015454,-0.016660,-0.026703,-0.019070,-0.025096,-0.019070,-0.007018,2
4,-0.174179,-0.127743,-0.104079,-0.063893,-0.054070,-0.008081,-0.053624,-0.021922,-0.059428,-0.045140,...,0.064253,0.008440,0.001742,-0.017904,-0.041568,-0.045140,-0.043801,0.017370,0.043267,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,-0.023708,-0.058982,-0.042015,-0.048712,-0.033085,-0.062554,-0.059875,-0.037103,-0.004062,-0.060768,...,-0.000937,0.073183,0.121405,0.162037,0.216956,0.171413,0.152660,0.054430,0.014244,2
800,-0.004062,-0.054070,-0.033085,-0.015225,0.038802,-0.014778,0.017370,-0.008081,-0.050945,-0.078181,...,-0.111223,-0.134887,-0.038889,-0.092916,-0.079521,-0.048266,0.036123,-0.035764,-0.094702,2
801,0.045500,-0.062554,0.040142,-0.128190,0.001296,0.005314,0.007547,-0.002723,0.026747,0.064699,...,0.102205,0.162930,0.080773,0.079434,0.204008,0.151767,0.128103,0.173646,0.377251,0
802,0.093817,0.097433,0.089800,0.080158,0.072525,0.046814,0.008248,0.033557,0.062482,0.026326,...,0.102655,0.081364,0.058866,0.050430,0.066499,0.083774,0.096228,0.102254,0.088595,4


In [14]:
x_sub_set=subject_466_tst_df.drop([3000],axis=1)

In [ ]:
x_sub_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.031299,-0.065233,0.074522,-0.027280,0.090150,-0.206774,0.006654,-0.084432,0.140158,-0.162570,...,-0.092470,-0.004062,0.038802,-0.030852,-0.022369,-0.033085,-0.023262,-0.080860,0.099526,-0.012992
1,-0.275376,-0.251674,-0.206680,-0.121914,-0.158472,-0.066073,-0.146821,-0.198645,-0.142804,-0.215920,...,-0.000188,0.036771,0.054849,0.039181,0.034361,0.014274,0.049627,0.029942,0.003829,-0.006616
2,0.115511,0.127965,0.159300,0.145641,0.166933,0.172959,0.168540,0.164522,0.164924,0.166531,...,0.022309,0.022710,0.048421,0.068508,0.059268,0.057259,0.068106,0.115913,0.111895,0.114306
3,0.005034,0.014676,0.048823,0.050430,0.053242,0.070517,0.107878,0.115913,0.103057,0.079355,...,0.042395,0.014274,0.003025,-0.015454,-0.016660,-0.026703,-0.019070,-0.025096,-0.019070,-0.007018
4,-0.174179,-0.127743,-0.104079,-0.063893,-0.054070,-0.008081,-0.053624,-0.021922,-0.059428,-0.045140,...,0.041928,0.064253,0.008440,0.001742,-0.017904,-0.041568,-0.045140,-0.043801,0.017370,0.043267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,-0.023708,-0.058982,-0.042015,-0.048712,-0.033085,-0.062554,-0.059875,-0.037103,-0.004062,-0.060768,...,0.000403,-0.000937,0.073183,0.121405,0.162037,0.216956,0.171413,0.152660,0.054430,0.014244
800,-0.004062,-0.054070,-0.033085,-0.015225,0.038802,-0.014778,0.017370,-0.008081,-0.050945,-0.078181,...,-0.114348,-0.111223,-0.134887,-0.038889,-0.092916,-0.079521,-0.048266,0.036123,-0.035764,-0.094702
801,0.045500,-0.062554,0.040142,-0.128190,0.001296,0.005314,0.007547,-0.002723,0.026747,0.064699,...,0.035677,0.102205,0.162930,0.080773,0.079434,0.204008,0.151767,0.128103,0.173646,0.377251
802,0.093817,0.097433,0.089800,0.080158,0.072525,0.046814,0.008248,0.033557,0.062482,0.026326,...,0.078551,0.102655,0.081364,0.058866,0.050430,0.066499,0.083774,0.096228,0.102254,0.088595


In [15]:
x_sub_set_rshp=np.array(x_sub_set).reshape(804,3000,1)

In [ ]:
x_sub_set_rshp.shape

(804, 3000, 1)

In [16]:
y_sub_dt=subject_466_tst_df[3000]

In [ ]:
y_sub_dt

0      0
1      0
2      2
3      2
4      0
      ..
799    2
800    2
801    0
802    4
803    1
Name: 3000, Length: 804, dtype: int64

In [ ]:
y_sub_dt.value_counts()

0    430
2    165
1    114
4     81
3     14
Name: 3000, dtype: int64

In [17]:
y_sub_dt_=np_utils.to_categorical(y_sub_dt)

In [18]:
y_sub_dt_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [19]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-12zclr40
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-12zclr40
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=618a03157f57718090f57203d19adf94c3cdded4bcec79ee3dd1b109e211e558
  Stored in directory: /tmp/pip-ephem-wheel-cache-u3simvzc/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


In [20]:
from keras_contrib.layers import CRF

In [21]:
from keras import optimizers, losses, activations, models
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from keras_contrib.layers import CRF

WINDOW_SIZE = 100

In [22]:
nclass = 5

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)

    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)

    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)

    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

Checking for individual model accuracies wrt to the test set

In [ ]:
loaded_model_shuf1=model_b(verbose=VBS)
loaded_model_shuf2=model_b(verbose=VBS)
loaded_model_shuf3=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
#dt_shuf1
checkpoint_path_load_model_30_epochs_shuf1 = "/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf1_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf1)
#latest_30_epochs_shuf1_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf1_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf1)

/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
#dt_shuf2
checkpoint_path_load_model_30_epochs_shuf2 = "/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf2_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf2)
#latest_300_epochs_shuf2_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf2_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf2)

/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt


In [ ]:
#dt_shuf3
checkpoint_path_load_model_30_epochs_shuf3 = "/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf3_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf3)
#latest_300_epochs_shuf3_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf3_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf3)

/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt


In [ ]:
loaded_model_shuf1.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
optimizer = Adam(lr=1e-4)

In [ ]:
loaded_model_shuf1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loaded_model_shuf1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
loaded_model_shuf2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
preds_m1 = loaded_model_shuf1.predict(x_sub_set_rshp)
preds_m1

array([[9.99465525e-01, 2.37199551e-04, 2.06680095e-04, 7.12667606e-05,
        1.93490032e-05],
       [9.44654047e-01, 4.12203483e-02, 1.20905675e-02, 3.84249463e-04,
        1.65084563e-03],
       [1.07659679e-02, 2.18521684e-01, 3.07646841e-01, 3.12327320e-05,
        4.63034332e-01],
       ...,
       [9.93129194e-01, 5.42415818e-03, 1.31458172e-03, 5.10602185e-05,
        8.09482226e-05],
       [2.79008411e-02, 2.51191080e-01, 2.43683271e-02, 1.95193898e-05,
        6.96520209e-01],
       [9.87767056e-02, 5.57351649e-01, 4.02215943e-02, 2.87857583e-05,
        3.03621262e-01]], dtype=float32)

In [ ]:
preds_m2 = loaded_model_shuf2.predict(x_sub_set_rshp)
preds_m2

array([[9.9974340e-01, 1.3789682e-04, 7.6487362e-05, 3.7464783e-05,
        4.7170806e-06],
       [7.3057419e-01, 2.1114990e-01, 4.9191367e-02, 1.0383294e-03,
        8.0461511e-03],
       [5.8859023e-03, 4.9366230e-01, 2.6582012e-01, 2.7518478e-05,
        2.3460421e-01],
       ...,
       [9.9180627e-01, 5.7971328e-03, 1.9504026e-03, 2.2797566e-04,
        2.1818664e-04],
       [3.4187943e-02, 5.8651555e-01, 1.7746437e-02, 1.2557785e-05,
        3.6153755e-01],
       [9.4199069e-02, 7.8312927e-01, 9.1897855e-03, 1.2401306e-05,
        1.1346939e-01]], dtype=float32)

In [ ]:
preds_m3 = loaded_model_shuf3.predict(x_sub_set_rshp)
preds_m3

array([[9.99078035e-01, 6.51075621e-04, 1.82575255e-04, 6.23187807e-05,
        2.59426561e-05],
       [8.58431816e-01, 1.01032205e-01, 3.09607144e-02, 2.44747195e-03,
        7.12782331e-03],
       [1.35049811e-02, 4.27130848e-01, 2.26462707e-01, 2.03586915e-05,
        3.32881093e-01],
       ...,
       [9.92386460e-01, 5.59665123e-03, 1.62274856e-03, 1.59900683e-05,
        3.78247467e-04],
       [2.66047306e-02, 2.53831625e-01, 3.08586750e-02, 1.39890553e-05,
        6.88690960e-01],
       [8.55390802e-02, 5.12522161e-01, 1.71313453e-02, 1.93336837e-05,
        3.84788096e-01]], dtype=float32)

In [ ]:
preds_m1_fin=np.argmax(preds_m1,axis=1)
preds_m2_fin=np.argmax(preds_m2,axis=1)
preds_m3_fin=np.argmax(preds_m3,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m1 = confusion_matrix(y_sub_dt, preds_m1_fin)
matrix_m1.diagonal()/matrix_m1.sum(axis=1)

array([0.98139535, 0.28070175, 0.86060606, 0.92857143, 0.5308642 ])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m2 = confusion_matrix(y_sub_dt, preds_m2_fin)
matrix_m2.diagonal()/matrix_m2.sum(axis=1)

array([0.95348837, 0.33333333, 0.84242424, 0.92857143, 0.27160494])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m3 = confusion_matrix(y_sub_dt, preds_m3_fin)
matrix_m3.diagonal()/matrix_m3.sum(axis=1)

array([0.98372093, 0.24561404, 0.75151515, 1.        , 0.43209877])

In [23]:
from sklearn.metrics import classification_report

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m1 = f1_score(y_sub_dt, preds_m1_fin, average="macro")
prec_m1 = precision_score(y_sub_dt, preds_m1_fin, average="macro")
recall_m1 = recall_score(y_sub_dt, preds_m1_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m1))
    print("Recall: {}".format(recall_m1))
    print("F1 score: {}".format(f1_m1))

    report_m1 = classification_report(y_sub_dt, preds_m1_fin, digits=6)
    print(report_m1)

Precision: 0.6933274511718017
Recall: 0.7164277579863056
F1 score: 0.6790635216102883
              precision    recall  f1-score   support

           0   0.862986  0.981395  0.918390       430
           1   0.561404  0.280702  0.374269       114
           2   0.825581  0.860606  0.842730       165
           3   0.500000  0.928571  0.650000        14
           4   0.716667  0.530864  0.609929        81

    accuracy                       0.810945       804
   macro avg   0.693327  0.716428  0.679064       804
weighted avg   0.791486  0.810945  0.789961       804



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m2 = f1_score(y_sub_dt, preds_m2_fin, average="macro")
prec_m2 = precision_score(y_sub_dt, preds_m2_fin, average="macro")
recall_m2 = recall_score(y_sub_dt, preds_m2_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m2))
    print("Recall: {}".format(recall_m2))
    print("F1 score: {}".format(f1_m2))

    report_m2 = classification_report(y_sub_dt, preds_m2_fin, digits=6)
    print(report_m2)

Precision: 0.7007566437258891
Recall: 0.6658844629387265
F1 score: 0.6540608691209318
              precision    recall  f1-score   support

           0   0.854167  0.953488  0.901099       430
           1   0.342342  0.333333  0.337778       114
           2   0.837349  0.842424  0.839879       165
           3   0.684211  0.928571  0.787879        14
           4   0.785714  0.271605  0.403670        81

    accuracy                       0.773632       804
   macro avg   0.700757  0.665884  0.654061       804
weighted avg   0.768287  0.773632  0.756576       804



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m3 = f1_score(y_sub_dt, preds_m3_fin, average="macro")
prec_m3 = precision_score(y_sub_dt, preds_m3_fin, average="macro")
recall_m3 = recall_score(y_sub_dt, preds_m3_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m3))
    print("Recall: {}".format(recall_m3))
    print("F1 score: {}".format(f1_m3))

    report_m3 = classification_report(y_sub_dt, preds_m3_fin, digits=6)
    print(report_m3)

Precision: 0.6554595819661446
Recall: 0.6825897764535055
F1 score: 0.6108545515921068
              precision    recall  f1-score   support

           0   0.831041  0.983721  0.900958       430
           1   0.500000  0.245614  0.329412       114
           2   0.821192  0.751515  0.784810       165
           3   0.311111  1.000000  0.474576        14
           4   0.813953  0.432099  0.564516        81

    accuracy                       0.776119       804
   macro avg   0.655460  0.682590  0.610855       804
weighted avg   0.771306  0.776119  0.754762       804



In [ ]:
check_pt_path="/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt"

In [ ]:
check_pt_path="/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt"

In [ ]:
#print(latest_teacher_30_epochs)
print(check_pt_path)

/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt


In [ ]:
optimizer=Adam(lr=1e-4)

In [24]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [25]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)


    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
m=get_model_1d_cnn()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170   

In [ ]:
hist_19_cnn = m.fit(x_transfer_set_rshp, y_transfer_, epochs=30)

Epoch 1/30
3648/3648 [==============================] - 31s 5ms/step - loss: 0.7925 - acc: 0.6820
Epoch 2/30
3648/3648 [==============================] - 20s 5ms/step - loss: 0.6126 - acc: 0.7484
Epoch 3/30
3648/3648 [==============================] - 20s 5ms/step - loss: 0.5489 - acc: 0.7729
Epoch 4/30
3648/3648 [==============================] - 20s 5ms/step - loss: 0.5135 - acc: 0.7884
Epoch 5/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4889 - acc: 0.7988
Epoch 6/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4717 - acc: 0.8086
Epoch 7/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4563 - acc: 0.8153
Epoch 8/30
3648/3648 [==============================] - 20s 5ms/step - loss: 0.4419 - acc: 0.8214
Epoch 9/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4299 - acc: 0.8267
Epoch 10/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4151 - acc: 0.8328
Epoch 11/30
3648/36

In [ ]:
accuracy_1d_cnn = m.evaluate(x_sub_set_rshp, y_sub_dt_)

26/26 [==============================] - 0s 3ms/step - loss: 0.6438 - acc: 0.7674


In [ ]:
preds_m = m.predict(x_sub_set_rshp)

In [ ]:
preds_m_fin=np.argmax(preds_m,axis=1)

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m = f1_score(y_sub_dt, preds_m_fin, average="macro")
prec_m = precision_score(y_sub_dt, preds_m_fin, average="macro")
recall_m = recall_score(y_sub_dt, preds_m_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m))
    print("Recall: {}".format(recall_m))
    print("F1 score: {}".format(f1_m))

    report = classification_report(y_sub_dt, preds_m_fin, digits=6)
    print(report)

Precision: 0.6362618053142892
Recall: 0.6465043157576817
F1 score: 0.5921971737692626
              precision    recall  f1-score   support

           0   0.814313  0.979070  0.889124       430
           1   0.531250  0.149123  0.232877       114
           2   0.759563  0.842424  0.798851       165
           3   0.448276  0.928571  0.604651        14
           4   0.627907  0.333333  0.435484        81

    accuracy                       0.767413       804
   macro avg   0.636262  0.646504  0.592197       804
weighted avg   0.737788  0.767413  0.726891       804



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m = confusion_matrix(y_sub_dt, preds_m_fin)
matrix_m.diagonal()/matrix_m.sum(axis=1)

array([0.97906977, 0.14912281, 0.84242424, 0.92857143, 0.33333333])

In [ ]:
cm_m = matrix_m.astype('float') / matrix_m.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_m.diagonal()

array([0.97906977, 0.14912281, 0.84242424, 0.92857143, 0.33333333])

# light weight model trained on transfer set + subject specific train set and testing on subject specific test set

In [26]:
subject_466_tr_df=pd.read_csv('/content/gdrive/My Drive/sub_466/x_train_sub.csv',header=None)

In [27]:
subject_466_tr_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.076141,0.125956,0.148855,0.136803,0.058063,0.028736,0.037976,0.042395,0.077346,0.074936,...,0.043601,0.064089,0.076944,0.076944,0.045207,0.012667,-0.021882,-0.038755,-0.033532,4
1,0.093014,0.083774,0.057661,0.030745,0.071320,0.129973,0.156488,0.149658,0.123546,0.106673,...,0.046011,0.074936,0.064892,0.117921,0.116716,0.097433,0.062482,0.025924,0.005034,4
2,-0.061654,-0.093391,-0.102631,-0.091382,-0.077321,-0.044379,0.002222,0.028335,-0.037550,-0.101827,...,0.095826,0.053242,0.023112,0.019095,0.013470,0.037976,0.044404,0.068106,0.078551,1
3,0.017370,0.049072,0.000849,-0.013885,-0.026834,0.073629,0.066039,0.019603,0.028533,0.021835,...,0.133014,0.101312,0.011565,0.018263,0.019156,0.042821,-0.033978,-0.088898,-0.026387,1
4,0.079355,0.074132,0.093014,0.079355,0.060473,0.044806,0.024317,0.038780,0.039985,0.035566,...,-0.048397,-0.047191,-0.050405,-0.037148,-0.042371,-0.066876,-0.074108,-0.068483,-0.069688,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,-0.050405,-0.033532,0.013069,0.030343,0.028335,0.016684,0.073731,0.016684,-0.012642,-0.014651,...,-0.054423,-0.057636,-0.064868,-0.035139,0.001418,0.055251,0.019496,-0.035943,-0.035943,0
3212,-0.006616,-0.006616,0.037173,0.071320,0.102655,0.082970,0.023514,0.014676,-0.006616,0.031147,...,0.141222,0.170548,0.170950,0.155684,0.146444,0.144034,0.141222,0.138008,0.160907,0
3213,-0.188914,0.020942,-0.246959,-0.030852,-0.049159,0.021835,0.018263,-0.022815,-0.025941,0.114707,...,-0.088898,-0.214811,0.074969,0.055769,-0.110776,-0.067465,-0.246066,-0.050498,-0.079521,0
3214,0.046011,0.063285,0.033557,-0.000590,0.007846,0.027933,0.023514,0.025121,0.034361,0.044806,...,0.019496,0.043199,0.046814,0.040387,0.033557,0.030745,0.014274,-0.038353,-0.041165,4


In [28]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [29]:
transfer_df_plus_sub_tr = transfer_df.append(subject_466_tr_df)

In [30]:
transfer_df_plus_sub_tr

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,-0.050405,-0.033532,0.013069,0.030343,0.028335,0.016684,0.073731,0.016684,-0.012642,-0.014651,...,-0.054423,-0.057636,-0.064868,-0.035139,0.001418,0.055251,0.019496,-0.035943,-0.035943,0
3212,-0.006616,-0.006616,0.037173,0.071320,0.102655,0.082970,0.023514,0.014676,-0.006616,0.031147,...,0.141222,0.170548,0.170950,0.155684,0.146444,0.144034,0.141222,0.138008,0.160907,0
3213,-0.188914,0.020942,-0.246959,-0.030852,-0.049159,0.021835,0.018263,-0.022815,-0.025941,0.114707,...,-0.088898,-0.214811,0.074969,0.055769,-0.110776,-0.067465,-0.246066,-0.050498,-0.079521,0
3214,0.046011,0.063285,0.033557,-0.000590,0.007846,0.027933,0.023514,0.025121,0.034361,0.044806,...,0.019496,0.043199,0.046814,0.040387,0.033557,0.030745,0.014274,-0.038353,-0.041165,4


In [31]:
x_transfer_df_plus_sub_tr=transfer_df_plus_sub_tr.drop([3000],axis=1)

In [32]:
x_transfer_df_plus_sub_tr

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,-0.050405,-0.033532,0.013069,0.030343,0.028335,0.016684,0.073731,0.016684,-0.012642,-0.014651,...,-0.002197,-0.054423,-0.057636,-0.064868,-0.035139,0.001418,0.055251,0.019496,-0.035943,-0.035943
3212,-0.006616,-0.006616,0.037173,0.071320,0.102655,0.082970,0.023514,0.014676,-0.006616,0.031147,...,0.131179,0.141222,0.170548,0.170950,0.155684,0.146444,0.144034,0.141222,0.138008,0.160907
3213,-0.188914,0.020942,-0.246959,-0.030852,-0.049159,0.021835,0.018263,-0.022815,-0.025941,0.114707,...,0.041481,-0.088898,-0.214811,0.074969,0.055769,-0.110776,-0.067465,-0.246066,-0.050498,-0.079521
3214,0.046011,0.063285,0.033557,-0.000590,0.007846,0.027933,0.023514,0.025121,0.034361,0.044806,...,0.000615,0.019496,0.043199,0.046814,0.040387,0.033557,0.030745,0.014274,-0.038353,-0.041165


In [33]:
y_transfer_df_plus_sub_tr = transfer_df_plus_sub_tr[3000]

In [34]:
y_transfer_df_plus_sub_tr

0       4
1       3
2       2
3       2
4       0
       ..
3211    0
3212    0
3213    0
3214    4
3215    0
Name: 3000, Length: 119934, dtype: int64

In [35]:
x_transfer_df_plus_sub_tr_rshp=np.array(x_transfer_df_plus_sub_tr).reshape(119934,3000,1)

In [36]:
x_transfer_df_plus_sub_tr_rshp.shape

(119934, 3000, 1)

In [37]:
y_transfer_df_plus_sub_tr_=np_utils.to_categorical(y_transfer_df_plus_sub_tr)

In [38]:
y_transfer_df_plus_sub_tr_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [68]:
light_wt_model=get_model_1d_cnn()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_16 (Conv1D)          (None, 2995, 4)           28        
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 1497, 4)          0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 1493, 4)           84        
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 746, 4)           0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 743, 10)           170 

In [69]:
hist = light_wt_model.fit(x_transfer_df_plus_sub_tr_rshp, y_transfer_df_plus_sub_tr_, epochs=30)

Epoch 1/30
3748/3748 [==============================] - 19s 4ms/step - loss: 0.7384 - acc: 0.6942
Epoch 2/30
3748/3748 [==============================] - 16s 4ms/step - loss: 0.5423 - acc: 0.7741
Epoch 3/30
3748/3748 [==============================] - 16s 4ms/step - loss: 0.4748 - acc: 0.8062
Epoch 4/30
3748/3748 [==============================] - 16s 4ms/step - loss: 0.4397 - acc: 0.8225
Epoch 5/30
3748/3748 [==============================] - 16s 4ms/step - loss: 0.4170 - acc: 0.8328
Epoch 6/30
3748/3748 [==============================] - 17s 4ms/step - loss: 0.4057 - acc: 0.8365
Epoch 7/30
3748/3748 [==============================] - 16s 4ms/step - loss: 0.3969 - acc: 0.8401
Epoch 8/30
3748/3748 [==============================] - 16s 4ms/step - loss: 0.3907 - acc: 0.8433
Epoch 9/30
3748/3748 [==============================] - 17s 4ms/step - loss: 0.3826 - acc: 0.8459
Epoch 10/30
3748/3748 [==============================] - 17s 4ms/step - loss: 0.3786 - acc: 0.8488
Epoch 11/30
3748/37

In [70]:
accuracy = light_wt_model.evaluate(x_sub_set_rshp, y_sub_dt_)

26/26 [==============================] - 0s 2ms/step - loss: 0.5161 - acc: 0.7687


In [71]:
preds_light_wt_model = light_wt_model.predict(x_sub_set_rshp)

26/26 [==============================] - 0s 2ms/step


In [72]:
preds_light_wt_model_fin=np.argmax(preds_light_wt_model,axis=1)

In [73]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_light_wt_model = f1_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
prec_light_wt_model = precision_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
recall_light_wt_model = recall_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_light_wt_model))
    print("Recall: {}".format(recall_light_wt_model))
    print("F1 score: {}".format(f1_light_wt_model))

    report = classification_report(y_sub_dt, preds_light_wt_model_fin, digits=6)
    print(report)

Precision: 0.6594866855474981
Recall: 0.6367063182975056
F1 score: 0.5782656417935115
              precision    recall  f1-score   support

           0   0.879570  0.951163  0.913966       430
           1   0.365079  0.403509  0.383333       114
           2   0.820809  0.860606  0.840237       165
           3   0.413793  0.857143  0.558140        14
           4   0.818182  0.111111  0.195652        81

    accuracy                       0.768657       804
   macro avg   0.659487  0.636706  0.578266       804
weighted avg   0.780265  0.768657  0.745033       804



In [75]:
from sklearn.metrics import confusion_matrix
matrix_light_wt_model = confusion_matrix(y_sub_dt, preds_light_wt_model_fin)
matrix_light_wt_model.diagonal()/matrix_light_wt_model.sum(axis=1)

array([0.95116279, 0.40350877, 0.86060606, 0.85714286, 0.11111111])

In [ ]:
import keras

In [ ]:
teacher_model=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_sub_set_rshp, y_sub_dt_)

26/26 [==============================] - 12s 7ms/step - loss: 0.9389 - accuracy: 0.8072


In [ ]:
preds_teacher = teacher_model.predict(x_sub_set_rshp)

In [ ]:
preds_teacher_fin=np.argmax(preds_teacher,axis=1)

In [ ]:
print(classification_report(y_sub_dt, preds_teacher_fin, digits=6))

              precision    recall  f1-score   support

           0   0.853955  0.979070  0.912243       430
           1   0.471429  0.289474  0.358696       114
           2   0.825843  0.890909  0.857143       165
           3   0.666667  0.857143  0.750000        14
           4   0.800000  0.444444  0.571429        81

    accuracy                       0.807214       804
   macro avg   0.723579  0.692208  0.689902       804
weighted avg   0.785250  0.807214  0.785286       804



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_sub_dt, preds_teacher_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.97906977, 0.28947368, 0.89090909, 0.85714286, 0.44444444])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm

array([[0.97906977, 0.02093023, 0.        , 0.        , 0.        ],
       [0.49122807, 0.28947368, 0.18421053, 0.        , 0.03508772],
       [0.01212121, 0.03030303, 0.89090909, 0.03636364, 0.03030303],
       [0.        , 0.        , 0.14285714, 0.85714286, 0.        ],
       [0.17283951, 0.28395062, 0.09876543, 0.        , 0.44444444]])

In [ ]:
cm.diagonal()

array([0.97906977, 0.28947368, 0.89090909, 0.85714286, 0.44444444])

In [ ]:
#student_model=get_model_cnn() #issue with unmatched shape
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=4,
)

# Distill teacher to student
distiller.fit(x_transfer_set_rshp, y_transfer_, epochs=40)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/40
3648/3648 [==============================] - 43s 8ms/step - categorical_accuracy: 0.6824 - student_loss: 0.7944 - distillation_loss: 0.0025
Epoch 2/40
3648/3648 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7607 - student_loss: 0.5808 - distillation_loss: 0.0018
Epoch 3/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.7853 - student_loss: 0.5245 - distillation_loss: 0.0016
Epoch 4/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.8101 - student_loss: 0.4700 - distillation_loss: 0.0014
Epoch 5/40
3648/3648 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8256 - student_loss: 0.4387 - distillation_loss: 0.0013
Epoch 6/40
3648/3648 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8331 - student_loss: 0.4202 - distillation_loss: 0.0013
Epoch 7/40
3648/3648 [==============================] - 29s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

26/26 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7600 - student_loss: 0.5685


In [ ]:
preds = student_model.predict(x_sub_set_rshp)

In [ ]:
preds

array([[9.9999964e-01, 2.1480402e-09, 1.3696576e-09, 2.3661912e-12,
        3.2631948e-07],
       [6.3198030e-01, 4.3314889e-02, 3.0685657e-01, 8.1088925e-03,
        9.7393878e-03],
       [3.3374850e-02, 3.2557622e-01, 3.3175558e-01, 1.2186460e-07,
        3.0929324e-01],
       ...,
       [9.9695694e-01, 3.0429207e-03, 6.8484191e-10, 6.8494869e-11,
        1.4104407e-07],
       [1.1937349e-02, 2.6336303e-01, 1.1618622e-01, 6.2868954e-10,
        6.0851341e-01],
       [2.4279298e-01, 5.9350961e-01, 1.6206075e-02, 6.3280392e-10,
        1.4749134e-01]], dtype=float32)

In [ ]:
preds_fin=np.argmax(preds,axis=1)

In [ ]:
preds_fin

array([0, 0, 2, 2, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0,
       0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 3, 2, 2, 4, 4, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 2, 2, 0, 0, 0, 1, 2, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 2, 0, 0, 4, 2, 0, 0, 0, 0, 0, 2,
       2, 2, 4, 0, 1, 0, 4, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 0, 2,
       2, 4, 0, 0, 0, 0, 2, 0, 0, 0, 4, 0, 0, 3, 2, 2, 0, 0, 0, 1, 2, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 2, 0, 0, 1,
       2, 0, 0, 2, 0, 3, 0, 4, 0, 0, 2, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0,
       0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, 0, 0, 2, 0, 2,
       0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 2,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0,
       2, 0, 2, 0, 3, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2,
       0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [ ]:
print(accuracy_score(y_sub_dt, preds_fin))
#print(precision_score(y_sub_dt, preds_fin))
#print(recall_score(y_sub_dt, preds_fin))
#print(f1_score(y_sub_dt, preds_fin))

0.7599502487562189


In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1 = f1_score(y_sub_dt, preds_fin, average="macro")
prec = precision_score(y_sub_dt, preds_fin, average="macro")
recall = recall_score(y_sub_dt, preds_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec))
    print("Recall: {}".format(recall))
    print("F1 score: {}".format(f1))

    report = classification_report(y_sub_dt, preds_fin, digits=6)
    print(report)

Precision: 0.6319675843869392
Recall: 0.6402043790864681
F1 score: 0.591159158492813
              precision    recall  f1-score   support

           0   0.819444  0.960465  0.884368       430
           1   0.387097  0.210526  0.272727       114
           2   0.774725  0.854545  0.812680       165
           3   0.464286  0.928571  0.619048        14
           4   0.714286  0.246914  0.366972        81

    accuracy                       0.759950       804
   macro avg   0.631968  0.640204  0.591159       804
weighted avg   0.732185  0.759950  0.726185       804



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_stu = confusion_matrix(y_sub_dt, preds_fin)
matrix_stu.diagonal()/matrix_stu.sum(axis=1)

array([0.96046512, 0.21052632, 0.85454545, 0.92857143, 0.24691358])

In [ ]:
cm_stu = matrix_stu.astype('float') / matrix_stu.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_stu.diagonal()

array([0.96046512, 0.21052632, 0.85454545, 0.92857143, 0.24691358])

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

26/26 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7600 - student_loss: 0.5685


In [ ]:
#Distiller's performance (student model's prformance)

#alpha=0.9,
#temperature=4

#30 epochs 85.57
#40 epochs 84.85
#50 epochs 83.88
#60 epochs 83.88

#alpha=0.9,
#temperature=5

#30 epochs 85.18
#40 epochs 85.41
#50 epochs 84.01
#60 epochs